# Scrape news and Analyse sentiments
This notebook shows an example of scraping news articles linked to specific traded companies and utilizing our predeployed sentiment analysis model server to predict the sentiment of the author towards said companies.

In [23]:
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 
import mlrun

## Environment

In [ ]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [ ]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [ ]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun:0.6.5-rc15"

## Function

In [ ]:
# nuclio: start-code

In [131]:
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import requests
import pandas as pd
import v3io_frames as v3f
from unicodedata import normalize
from datetime import datetime
import re
import os
import mlrun.feature_store as fs
import mlrun
from mlrun.datastore.targets import ParquetTarget

In [132]:
def get_stock_news_page(stock_string):
    request = Request('https://www.investing.com/equities/' + stock_string + '-news', headers={"User-Agent": "Mozilla/5.0"})
    content = urlopen(request).read()
    return bs(content, 'html.parser')

def get_internal_article_links(page):
    news = page.find_all('div', attrs={'class': 'mediumTitle1'})[1]
    articles = news.find_all('article', attrs={'class': 'js-article-item articleItem'})
    return ['https://www.investing.com' + a.find('a').attrs['href'] for a in articles]

def get_article_page(article_link):
    request = Request(article_link, headers={"User-Agent": "Mozilla/5.0"})
    content = urlopen(request).read()
    return bs(content, 'html.parser')

def clean_paragraph(paragraph):
    paragraph = re.sub(r'\(http\S+', '', paragraph)
    paragraph = re.sub(r'\([A-Z]+:[A-Z]+\)', '', paragraph)
    paragraph = re.sub(r'[\n\t\s\']', ' ', paragraph)
    return normalize('NFKD', paragraph)    

def extract_text(article_page):
    text_tag = article_page.find('div', attrs={'class': 'WYSIWYG articlePage'})
    paragraphs = text_tag.find_all('p')
    text = '\n'.join([clean_paragraph(p.get_text()) for p in paragraphs[:-1]])
    return text

import json
def get_publish_time(article):
    tag = article.find('script',{"type" : "application/ld+json"}).contents[0]
    tag_dict = json.loads(str(tag))
    dateModified = tag_dict["dateModified"]
    return datetime.strftime(datetime.strptime(dateModified, '%Y-%m-%d %H:%M:%S'), '%Y-%m-%d %H:%M:%S')

def get_score(paragraph_scores):
    return sum([score - 1 for score in paragraph_scores['outputs']]) / len(paragraph_scores)

def get_article_scores(context, articles, endpoint):
    scores = [] 
    endpoint = endpoint + "/v2/models/model1/predict"
    for i, article in enumerate(articles):
        context.logger.info(f'getting score for article {i + 1}\\{len(articles)}')
        event_data = {'inputs': article.split('\n')}
        resp = requests.put(endpoint, json=json.dumps(event_data))
        scores.append(get_score(json.loads(resp.text)))
    return scores

def construct_dataframe(sentiments, items,times):
    tickers = [x[0] for x in items]
    stock_sent = pd.DataFrame({"symbol": tickers, "sentiment": sentiments, "last_reaction": times})
    return stock_sent

In [134]:
def init_context(context):
    context.logger.info("init news reader context")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project = context.PROJECT_NAME)
    
    context.logger.info(os.environ)
    
    # Declaring feature set
    stocks_sent_set = fs.FeatureSet("news", entities=[fs.Entity("symbol")],timestamp_key="Datetime")
    stocks_sent_set.set_targets(targets=[ParquetTarget(name="news",partitioned=True,time_partitioning_granularity="day")]
                                ,with_defaults=False)
    setattr(context, 'stock_feature_set', stocks_sent_set)
    
#     # Add aggregation 
#     context.stock_feature_set.add_aggregation("Sentiments","Sentiment",["min","max"],["1d"])
    
    # saving timestamps to know what to ingest
    last_trade_times = {}
    setattr(context, 'last_trade_times', last_trade_times)
    
    v3io_framesd = os.getenv('V3IO_FRAMESD', 'framesd:8081')
    token = os.getenv('TOKEN', '')
    client = v3f.Client(v3io_framesd, container=os.getenv('V3IO_CONTAINER', 'users'), token=token)
    setattr(context, 'v3c', client)

    setattr(context, 'stocks_stream', os.getenv('STOCKS_STREAM', os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream'))
    context.v3c.create(backend='stream', table=context.stocks_stream, if_exists=1)

    setattr(context, 'stocks_tsdb', os.getenv('STOCKS_TSDB_TABLE', os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb'))
    context.v3c.create(backend='tsdb', table=context.stocks_tsdb, rate='1/s', if_exists=1)

    setattr(context, 'sentiment_model_endpoint',
            os.getenv('SENTIMENT_MODEL_ENDPOINT', 'http://default-tenant.app.dev39.lab.iguazeng.com:30856')) # in the '' should be the model endpoint
    context.logger.info(f"set sentiment_model_endpoint {context.sentiment_model_endpoint}")
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_kv', os.getenv('STOCKS_KV', os.getenv('V3IO_USERNAME') + '/stocks/stocks_kv'))
    context.logger.info('end init context')

In [4]:
def handler(context, event):
    context.logger.info(f'Getting news about {context.sym_to_url}')
    syms = []
    contents = []
    links = []
    times = []
    sentiments = []
    all_records = []
    for sym, url_string in list(context.sym_to_url.items()):
        context.logger.info(f'Getting news about {sym}')
        news_page = get_stock_news_page(url_string)
        article_links = get_internal_article_links(news_page)
        article_pages = [get_article_page(link) for link in article_links]
        articles = [extract_text(article_page) for article_page in article_pages]
        curr_sentiments = get_article_scores(context, articles, context.sentiment_model_endpoint)
        curr_times = [get_publish_time(article_page) for article_page in article_pages]
        sentiments += curr_sentiments
        times += curr_times
        time = datetime.strptime(curr_times[0],"%Y-%m-%d %H:%M:%S")
        last = context.last_trade_times.get(sym)
        if not last:
            last = datetime(1990,1,1)
        for article, link, sentiment, time in zip(articles, article_links, curr_sentiments, curr_times):
            record = {
                'content': article,
                'time': time,
                'symbol': sym,
                'link': link,
                'sentiment': sentiment
            }
            context.v3c.execute('stream', context.stocks_stream, 'put', args={'data': json.dumps(record)})
            timestamped_record = record.copy()
            timestamped_record["time"] = datetime.strptime(record["time"],"%Y-%m-%d %H:%M:%S")
            if(last):
                if(timestamped_record.get("time")>last):
                    all_records.append(timestamped_record)
                    context.last_trade_times[sym] = timestamped_record.get("time")
            else:
                all_records.append(timestamped_record)
                context.last_trade_times[sym] = timestamped_record.get("time")
            syms.append(sym)
            contents.append(article)
            links.append(link)
            
        context.v3c.execute('kv', context.stocks_kv, command='update', args={'key': sym,
                                                                         'expression': f"SET sentiment='{sentiments[-1]}';last_reaction='{times[-1]}'"})
    all_records = pd.DataFrame(all_records)
    if(all_records.shape[0]>0):
        all_records.columns = ["Content","Datetime","symbol","Link","Sentiment"]
        # Localizing the datetime
        all_records["Datetime"] = all_records["Datetime"].dt.tz_localize('UTC')
        # writing to featureset
        context.logger.info(f"Writing new dataframe with shape {all_records.shape} to feature store")
        fs.ingest(context.stock_feature_set, all_records, infer_options=fs.InferOptions.default(),
                 overwrite=False)
    else:
        context.logger.info("No new data to ingest")

    if len(sentiments) > 0:
        df = pd.DataFrame.from_dict({'sentiment': sentiments,
                                     'time': times,
                                     'symbol': syms})
        df = df.set_index(['time', 'symbol'])
        df.index = df.index.set_levels([pd.to_datetime(df.index.levels[0]), df.index.levels[1]])
        df = df.sort_index(level=0, axis=0)
        context.v3c.write(backend='tsdb', table=context.stocks_tsdb, dfs=df)

In [136]:
# nuclio: end-code

## Test locally

In [ ]:
init_context(context)

In [128]:
from nuclio import Event
event = Event()

In [ ]:
s = handler(context, event)

## Deploy to cluster

In [5]:
from mlrun import code_to_function,auto_mount
import os
import nuclio
# Export bare function
fn = code_to_function('news-reader',
                      handler='handler')
fn.export('02-read-news.yaml')

# Set parameters for current deployment
fn.add_trigger('cron', nuclio.triggers.CronTrigger('10s'))
fn.set_envs({'V3IO_CONTAINER': 'users',
             'STOCKS_STREAM': os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream',
             'STOCKS_TSDB_TABLE': os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb',
             'SENTIMENT_MODEL_ENDPOINT': 'http://default-tenant.app.dev39.lab.iguazeng.com:31437', # this is a temporary endopoint, make sure you insert the right endpoint when running test
             'PROJECT_NAME' :"stocks-" + os.getenv('V3IO_USERNAME')})
fn.spec.max_replicas = 1

> 2021-07-12 12:27:08,435 [info] function spec saved to path: 02-read-news.yaml


In [6]:
addr = fn.deploy(project= "stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-07-12 12:27:08,442 [info] Starting remote function deploy
2021-07-12 12:27:08  (info) Deploying function
2021-07-12 12:27:08  (info) Building
2021-07-12 12:27:09  (info) Staging files and preparing base images
2021-07-12 12:27:09  (info) Building processor image
2021-07-12 12:27:13  (info) Build complete
2021-07-12 12:27:19  (info) Function deploy complete
> 2021-07-12 12:27:20,154 [info] function deployed, address=default-tenant.app.dev39.lab.iguazeng.com:30639


In [7]:
fn.run()

> 2021-07-12 12:27:21,666 [info] starting run news-reader-handler uid=f5215836e36842deb25e9e09ba286f4c DB=http://mlrun-api:8080
2021-07-12 12:27:22.034586  info   logging run results to: http://mlrun-api:8080  worker_id=0

2021-07-12 12:27:22.034586  info   logging run results to: http://mlrun-api:8080  worker_id=0

final state: error
> 2021-07-12 12:27:22,344 [warning] run ended with state error


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
stocks-dani,...ba286f4c,0,Jul 12 12:27:22,error,news-reader-handler,v3io_user=danikind=remoteowner=danihost=nuclio-stocks-dani-news-reader-9c48dbd78-sfql6,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run f5215836e36842deb25e9e09ba286f4c --project stocks-dani , !mlrun logs f5215836e36842deb25e9e09ba286f4c --project stocks-dani
> 2021-07-12 12:27:22,632 [info] run executed, status=error


RunError: 'MLClientCtx' object has no attribute 'mlrun_handler'

In [158]:
!curl {addr}

Exception caught in handler - "Expecting value: line 1 column 1 (char 0)": Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 114, in serve_requests
    self._handle_event(event)
  File "/opt/nuclio/_nuclio_wrapper.py", line 262, in _handle_event
    entrypoint_output = self._entrypoint(self._context, event)
  File "/opt/nuclio/02-read-news.py", line 116, in handler
    curr_sentiments = get_article_scores(context, articles, context.sentiment_model_endpoint)
  File "/opt/nuclio/02-read-news.py", line 60, in get_article_scores
    scores.append(get_score(json.loads(resp.text)))
  File "/usr/local/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/local/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/local/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.dec

in case running this notebook without running the entire project (means sentiment analysis serving function isn't depolyed)
run the cells below to deploy the sentiment analysis serving function, then copy the given url after depolyment to the sentiment analysis endpoint in the set_enviornemt section above

In [9]:
# Deploying sentiment analysis serving function
import mlrun
fn = mlrun.import_function(url = "hub://sentiment_analysis_serving")
fn.apply(mlrun.auto_mount())
fn.add_model(model_path="/User/test/demos/stock-analysis/models/model.pt",key="model1",class_name="SentimentClassifierServing")
addr = fn.deploy(project="stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-07-12 12:28:28,700 [info] Starting remote function deploy
2021-07-12 12:28:28  (info) Deploying function
2021-07-12 12:28:28  (info) Building
2021-07-12 12:28:28  (info) Staging files and preparing base images
2021-07-12 12:28:28  (info) Building processor image
2021-07-12 12:28:30  (info) Build complete
2021-07-12 12:29:08  (info) Function deploy complete
> 2021-07-12 12:29:08,944 [info] function deployed, address=default-tenant.app.dev39.lab.iguazeng.com:31437


In [126]:
!curl {addr}

In [10]:
addr

'http://default-tenant.app.dev39.lab.iguazeng.com:31437'